In [ ]:
import json
import datetime
import string
import pandas as pd
import requests

In [ ]:
def download_jsons(list_of_uuids):
    
def to_datetime(string_date):#converts a string from the format found in the json file to a datetime timedelta object
    temp = string_date
    temp = temp.replace("PT","")
    temp = temp.replace("H",":")
    temp = temp.replace("M",":")

    if(len(temp)<=10):
        temp = temp[:-1]
    else:
        temp = temp[:-2]
    #print(temp)
    try:
        timestamp = datetime.datetime.strptime(temp, "%H:%M:%S.%f")
    except:
        try:
            timestamp = datetime.datetime.strptime(temp, "%M:%S.%f")
        except:
            try:
                timestamp = datetime.datetime.strptime(temp, "%S.%f")
            except:
                timestamp = datetime.datetime.strptime(temp, "%M:%S")
    return datetime.timedelta(hours = timestamp.hour,  minutes = timestamp.minute, seconds = timestamp.second, microseconds = timestamp.microsecond)
    



def json_to_csv(filename, name_of_new_file = "placeholder.xlsx"):#creates 2 csv files based on the stream json files
    with open(filename, 'r') as f:
            datastore = json.load(f)
            
    datastore_length = len(datastore['value'])
    actor_ids = list()
    start_time = list()
    end_time = list()
    
    for i in range(0,datastore_length):
        y_length = 0
        for y in datastore['value'][i]['events']:
            y_length = len(y['start'])
            start_time.append(y['start'])
            end_time.append(y['end'])
            actor_ids.append(y['eventData']['personId'])
            
    df = pd.DataFrame(list(zip(actor_ids, start_time, end_time)), 
               columns =['PersonId', 'Start_time','End_time']) 
    df['Start_time'] = df['Start_time'].map(lambda x: to_datetime(x))
    df['End_time'] = df['End_time'].map(lambda x: to_datetime(x))
    df['Total_time'] = df['End_time']-df['Start_time']
    df.sort_values(by=['Start_time'])
    #create second sheet of excel file
    all_ids = list()
    image_urls = list()
    for i in range(0,datastore_length):
        all_ids.append(datastore['value'][i]['id'])
        image_urls.append(datastore['value'][i]['imageUrl'])
    total_screentime = list()
    for i in all_ids:
        total_screentime.append(sum(df[df['PersonId']==i]['Total_time'].tolist(),datetime.timedelta()))
    #print(total_screentime)
    df2 = pd.DataFrame(list(zip(all_ids, image_urls,total_screentime)), columns = ['ActorId', 'Image_url','Total_ScreenTime'])
    df.to_csv(name_of_new_file)
    df2.to_csv(name_of_new_file+"_aggregated")


In [ ]:
#to save the movie to csv just run the json_to_csv function, the first parameter is the filename of the json file
#you want to convert and the second parameter is the new filename without the .csv
json_to_csv("JSON/dirty_harry_1.json","dirty_harry_1_screentime")
json_to_csv("JSON/dirty_harry_2.json","dirty_harry_2_screentime")
json_to_csv("JSON/dirty_harry_3.json","dirty_harry_3_screentime")
json_to_csv("JSON/dirty_harry_4.json","dirty_harry_4_screentime")
json_to_csv("JSON/dirty_harry_5.json","dirty_harry_5_screentime")